In [28]:
from scipy.optimize import broyden1 as solve
import numpy as np
from suspension import Suspension
from vehicle import Vehicle
from aerodynamics import Aerodynamics

In [49]:
# parameters
mass = 272
sprung_inertia = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
unsprung_inertia = np.array([[1, 1, 1], [1, 1, 1], [1, 1, 1]])

# sweep
rotational_velocities = np.array([0, 0, 1])
translation_velocities = np.array([1, 1, 0])

In [30]:
def car_loads(translation_accelerations, rotational_accelerations, roll, pitch, heave):
    # aero loads is roll/heave/pitch dependent
    # tire loads is aero force dependent
    # force/moment is aero & tire load dependent
    return translation_accelerations

In [56]:
def DOF6_motion_residuals(x, vehicle):
    x_double_dot, y_double_dot, yaw_acceleration, roll, pitch, ride_height = x
    translation_accelerations = np.array([x_double_dot, y_double_dot, 0])
    rotational_accelerations = np.array([0, 0, yaw_acceleration])

    forces, moments = vehicle.get_loads(x_double_dot, y_double_dot, yaw_acceleration, roll, pitch, ride_height)
    residuals_translation = mass * (translation_accelerations + np.cross(rotational_velocities, translation_velocities)) - forces
    residuals_translation[2] = 0 # TODO: REMOVEEE
    residuals_rotation = [0, 0, 0]
    #residuals_rotation = sprung_inertia.dot(rotational_accelerations) + np.cross(rotational_velocities, sprung_inertia.dot(rotational_velocities)) -\
    #    + unsprung_inertia.dot(rotational_accelerations) - moments
    return np.array([*residuals_translation, *residuals_rotation])

In [32]:
#translation_accelerations, rotational_accelerations = [1, 1, 1], [1, 1, 1]
# x = [translation_accelerations, rotational_accelerations]

#     specific_residual_func = lambda x: DOF6_motion_residuals(x, body_slip, steered_angle, velocity, yaw_rate)
#     solve(specific_residual_func, x, verbose=True)

In [57]:
# x_double_dot, y_double_dot, yaw_acceleration, roll, pitch, ride_height
x = [0, 0, 0, 0, 0, 0]

suspension = Suspension()
aero = Aerodynamics()
vehicle = Vehicle(suspension, aero)

vehicle.state.body_slip = 0
vehicle.state.steered_angle = 0
vehicle.state.x_dot = 30
vehicle.state.yaw_rate = 0

specific_residual_func = lambda x: DOF6_motion_residuals(x, vehicle)
solve(specific_residual_func, x)

array([-2.06430147, -1.        ,  0.        ,  0.        ,  0.        ,
        0.        ])